## 財務データ

### 財務諸表

> 財務諸表（ざいむしょひょう、financial statements）は、企業が利害関係者に対して一定期間の経営成績や財務状態等を明らかにするために複式簿記に基づき作成される書類である。日常用語としては、決算書（又は決算報告書）と呼ばれている。

> 日本における会計基準では、
>
> - 貸借対照表（B/S）
> - 損益計算書（P/L）
> - キャッシュ・フロー計算書（C/F）
> - 株主資本等変動計算書（S/S）
> 
> が財務諸表に含まれる。

「財務諸表」（2020年10月20日 (火) 04:19　UTCの版）『ウィキペディア日本語版』。

https://ja.wikipedia.org/wiki/財務諸表

### 有価証券報告書


> 有価証券報告書（ゆうかしょうけんほうこくしょ）とは、金融商品取引法で規定されている、事業年度ごとに作成する企業内容の外部への開示資料である。略して有報（ゆうほう）と呼ばれることもある。

> 次のような株式会社には、各事業年度終了後、3か月以内の金融庁への提出が義務づけられている。
> - 金融商品取引所（証券取引所）に株式公開している会社
> - 店頭登録している株式の発行会社
> - 有価証券届出書提出会社 - 有価証券届出書とは、1億円以上の有価証券（株券や社債券など）の募集（新規発行）または売出しを行う際に、有価証券の発行者が金融商品取引法第4条・5条に基づき、内閣総理大臣（窓口は財務局）に提出することが義務づけられている書類。発行する会社の営業状況や事業の内容、および有価証券の発行条件などが記載されている。
> - 過去5年間において、事業年度末日時点の株券もしくは優先出資証券の保有者数が1000人以上となったことがある会社（ただし、資本金5億円未満の会社を除く）

「有価証券報告書」（2020年8月2日 (日) 02:01　UTCの版）『ウィキペディア日本語版』。

https://ja.wikipedia.org/wiki/有価証券報告書

### EDINET

有価証券報告書等の開示書類を閲覧するサイト

https://disclosure.edinet-fsa.go.jp

- 有価証券報告書はXBRL形式(タクソノミ)で提供される
- XBRLの構造は非常に複雑で扱いにくいため、今回は使わない

### バフェット・コード

https://www.buffett-code.com

- ワンストップで効率的な企業分析ができる無料のツール
- APIが提供されている
    - [BuffettCode API](https://docs.buffett-code.com)
    - 利用に際してはプレミアム会員になる必要がある
    - テスト用に使えるAPI Keyがある(ティッカーの末尾が"01"で終わる銘柄が取得できる)

### APIクライアント

[Python で Web API/SDK を使ってみる勉強会 #3](https://bizpy.connpass.com/event/186245/) の汎用のHTTPクライアントを利用する

https://github.com/t2y/python-study/tree/master/BizPy/webapi/20200909

In [3]:
import requests

class HttpClient:
    def __init__(self, headers={}):
        self.headers = headers

    def _handle_response(self, response):
        if response.ok:
            return response.json()
        else:
            # show extra information to confirm error
            # ColabではPythonのバージョンが古いためデバッグ用の記法は使えない
            # print(f"{response.status_code=}, {response.reason=}")
            print(f"{response.status_code}, {response.reason}")
            raise response.raise_for_status()

    def get(self, url, params):
        response = requests.get(url, headers=self.headers, params=params)
        return self._handle_response(response)

API Keyは下記の `env.py` ファイルを用意した前提としている


```python
api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
```

In [4]:
from env import api_key

client = HttpClient(headers={"x-api-key": api_key})

下記のデータを取得

- 証券コード: 1301
- 2018年第1四半期から2018年第2四半期まで

In [5]:
data = client.get(
    "https://api.buffett-code.com/api/v2/quarter",
    params={"tickers": "1301", "from": "2018Q1", "to": "2018Q2"},
)

In [6]:
data.keys()

dict_keys(['1301', 'column_description'])

1301を参照

In [7]:
data["1301"]

[{'company_name': '株式会社\u3000極洋',
  'ceo_name': '代表取締役社長\u3000井\u3000上\u3000誠',
  'headquarters_address': '東京都港区赤坂三丁目３番５号',
  'end_date': '2018-06-30T00:00:00.000Z',
  'updated_date': '2018-08-10T13:13:00.000Z',
  'edinet_updated_date': '2018-08-10T13:13:00.000Z',
  'tdnet_updated_date': '2018-08-03T11:30:00.000Z',
  'edinet_title': '【E00012】株式会社\u3000極洋 四半期報告書－第96期第1四半期(平成30年4月1日－平成30年6月30日)',
  'tdnet_title': '【13010】株式会社\u3000極洋 平成31年3月期\u3000第1四半期決算短信〔日本基準〕(連結)',
  'accounting_standard': 'Japan GAAP',
  'num_of_shares': 10784762.0,
  'issued_share_num': 10928283.0,
  'treasury_stock_num': 143521.0,
  'assets': 112367000000.0,
  'current_assets': 87022000000.0,
  'cash_and_deposits': 4978000000.0,
  'trade_receivables': 33951000000.0,
  'notes_accounts_receivable': 33951000000.0,
  'notes_receivable': None,
  'accounts_receivable': None,
  'current_securities': None,
  'inventories': 46045000000.0,
  'merchandise': None,
  'work_in_process': None,
  'raw_materials_and_supplies': Non

列データの概要

In [11]:
import pandas as pd

df = pd.DataFrame.from_dict(data["column_description"])
display_rows = pd.get_option("display.max_rows")  # pandasで表示する行数
pd.set_option("display.max_rows", len(df.T))
df.T

,name_jp,unit
company_name,社名,なし
ceo_name,代表者名,なし
headquarters_address,所在地,なし
end_date,期末日,日付
updated_date,更新日,日付
edinet_updated_date,edinet発行日,日付
tdnet_updated_date,tdnet発行日,日付
edinet_title,edinet開示資料名,なし
tdnet_title,tdnet開示資料名,なし
accounting_standard,会計基準,なし


In [12]:
pd.set_option("display.max_rows", display_rows)

データを保存

In [14]:
with open("data/1301.json", "w") as f:
    